 # Categorización de publicaciones de productos de Mercado Libre

 Autores: Maximiliano Tejerina, Eduardo Barseghian, Benjamín Ocampo

## Funciones y Constantes

In [ ]:
import pandas as pd
import numpy
import nltk
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download("stopwords")
nltk.download('punkt')

stopwords = \
    set(nltk.corpus.stopwords.words("spanish")) | \
    set(nltk.corpus.stopwords.words("portuguese"))


def remove_unimportant_words(s):
    """
    Removes from the string @s all the stopwords, digits, and special chars
    """
    special_chars = "-.+,[@_!#$%^&*()<>?/\|}{~:]"
    digits = "0123456789"
    invalid_chars = special_chars + digits

    reduced_title = ''.join(c for c in s if not c in invalid_chars)

    reduced_title = ' '.join(w.lower() for w in word_tokenize(reduced_title)
                             if not w.lower() in stopwords)
    return reduced_title


def expand_contractions(s):
    title = s
    contractions = {" c/u ": "cada uno", " p/": "para", " c/": "con"}
    for key, value in contractions.items():
        title = title.split(key)
        title = value.join(title)
    return title

def prepare_tokenizer(words):
    """
    funcion que genera un vocabulario, toma una lista de palabras.
    retorna una lista de palabras tokenizadas.
    """
    t = Tokenizer(filters='-.+,[@_!#$%^&*()<>?/\|}{~:]0123456789', lower=True)
    t.fit_on_texts(words)
    encoded_docs = t.texts_to_sequences(words)
    return pad_sequences(encoded_docs)

In [ ]:
URL = "https://www.famaf.unc.edu.ar/~nocampo043/ml_challenge2019_dataset.csv"
df = pd.read_csv(URL)

## Limpieza de Texto

## Tokenización y Secuencias
TODO: Explicar que hace la función o como se realiza el encoding de los
títulos.

In [ ]:
encoded_titles = prepare_tokenizer(df.title)

## Label Encoding

In [ ]:
le = preprocessing.LabelEncoder()
encoded_labels = le.fit_transform(df["category"])

## Word Embeddings